# Calls RAWG API to get game info

In [33]:
import requests 
import json 
from bs4 import BeautifulSoup

class RAWG:
    def __init__(self):
        self.base_url = 'https://api.rawg.io/api'
        self.session = requests.Session()

    def get_game(self, name):
        game = {}
        
        keys = ['Name', 'RawgID', 'Metacritic', 'Presence', 'Platform', 
                'Storage', 'RatingsBreakdown', 'ReleaseDate', 
                'Achievements', 'Publisher', 'Description'] 

        values = []

        id = self.__request_id(name)     
        res = self.__request_game(id)
        
        game['Name'] = res['name']
        game['RawgID'] = res['id']
        game['Metacritic'] = res['metacritic']
        game['Presence'] = self.__parse_precense(res)
        game['Presence'] = self.__score_precense(game['Presence'])
        game['Platform'] = self.__parse_platforms(res['platforms'])
        # storage = 
        # ratings_breakdown = 
        # release_date = 
        # achievements = 
        # publisher = 
        game['Description'] = self.__parse_description(res['description'])
        
        return game

    def __request_id(self, name):
        params = { 'search': name }
        res = self.session.get(f'{self.base_url}/games', params=params)
        res = json.loads(res.content)
        return res['results'][0]['id']

    def __request_game(self, id):
        res = self.session.get(f'{self.base_url}/games/{id}')
        res = json.loads(res.content)
        return res
    
    def __parse_description(self, description):
        soup = BeautifulSoup(description)
        return soup.get_text()

    def __parse_precense(self, social_media):
        keys = [ 'reddit_count', 'twitch_count', 'youtube_count', 
                'reviews_text_count', 'ratings_count', 'suggestions_count']
        vals = []

        for key in keys:
            vals.append(social_media[key])
        return dict(zip(keys, vals))
            
    def __score_precense(self, precense):
        score = 0
        for site in precense:
            score += precense[site]
        return score

    def __parse_platforms(self, platforms):
        platforms = [ platform['platform']['name'] for platform in platforms ]
        return ', '.join(platforms )


## Test

In [35]:
# calls api and returns game info
rawg = RAWG()

rawg.get_game('Crash Bandicoot')

[{'platform': {'id': 21, 'name': 'Android', 'slug': 'android', 'image': None, 'year_end': None, 'year_start': None, 'games_count': 30317, 'image_background': 'https://media.rawg.io/media/screenshots/124/124dd314f9effb6cc153589db7975e9a.jpg'}, 'released_at': '1996-09-09', 'requirements': None}, {'platform': {'id': 27, 'name': 'PlayStation', 'slug': 'playstation1', 'image': None, 'year_end': None, 'year_start': None, 'games_count': 1501, 'image_background': 'https://media.rawg.io/media/games/017/01778e235ab9478d18e8b53f855346a2.jpg'}, 'released_at': '1996-09-09', 'requirements': None}, {'platform': {'id': 19, 'name': 'PS Vita', 'slug': 'ps-vita', 'image': None, 'year_end': None, 'year_start': None, 'games_count': 1913, 'image_background': 'https://media.rawg.io/media/games/ffe/ffed87105b14f5beff72ff44a7793fd5.jpg'}, 'released_at': '1996-09-09', 'requirements': None}, {'platform': {'id': 16, 'name': 'PlayStation 3', 'slug': 'playstation3', 'image': None, 'year_end': None, 'year_start': No

{'Description': "Crash Bandicoot is the first installment in the 16 titles-long series. While the game set features various genres: races, beat'em'ups and platformers, the first part concentrates on platforming style. The game is developed by Naughty Dog who is currently famous for making Jax and Daxter, Uncharted and The Last of Us series. \nThe game features self-titled bandicoot protagonist trying to get past the obstacles while crashing enemies by jumping and circling them and collecting fruits and masks that give him various bonuses. All levels emphasize running against or into the screen. \nCrash Bandicoot tells a story of a mutated bandicoot, trapped on Wumpa islands. He fights evil corporation сalled Doctor Neo Cortex which turns every living being into the beasts fighting against each other. The game features two ending — usual and true. With latter ending achieved by collecting all of the gems namely clearing the level without deaths and collecting every bonus there is to gra